In [31]:
api_key = "RGAPI-990cb574-183d-4308-b0c1-9e85a45c4aae"

In [32]:
import requests

In [33]:
## récupère puis stocke le PUUID du summoner

data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ 'AEGIS EIKA' +'?api_key='+api_key).json()
puuid_summoner = data_summoner.get("puuid")
id_summoner = data_summoner.get("id")
#print("puuid =", puuid_summoner, "\nid =", id_summoner)

## récupère les matchs (les derniers) du summoner (ici 5 matchs)
## ressource utile les ID des queues : https://static.developer.riotgames.com/docs/lol/queues.json
## récupère la division, le tier et les LP du compte

matches_id = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid_summoner +'/ids?queue='+'420'+'&start=0&count=5&api_key='+api_key).json()
data_rank = requests.get('https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/'+ id_summoner +'?api_key='+api_key).json()
print(data_rank[0]['tier'],data_rank[0]['rank']," : ",data_rank[0]['leaguePoints']) 




CHALLENGER I  :  849


In [34]:
##vérifie le résultat des game avec l'ID du match et le PUUID du summoner 

win = 0
loose = 0
for match_id in matches_id:
    match_info = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+ match_id +"?api_key=" + api_key).json()
    list_player = []
    for player in (match_info['metadata']['participants']):
        list_player.append(player)
    position_player = list_player.index(puuid_summoner)
    if match_info['info']['participants'][position_player]['puuid'] == puuid_summoner:
            if match_info['info']['participants'][position_player]['win'] == True:
                print("WIN")
                win += 1
            else:
                print("LOOSE")
                loose += 1
print("WIN =",win,"\nLOOSE =",loose)

WIN
WIN
WIN
LOOSE
WIN
WIN = 4 
LOOSE = 1


In [35]:
##vérifie le résultat des game avec l'ID du match et le PUUID du summoner 

win = 0
loose = 0
for match_id in matches_id:
    match_info = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+ match_id +"?api_key=" + api_key).json()
    
    for player in (match_info['info']['participants']):
        if player['puuid'] == puuid_summoner:
            if player['win'] == True:
                print("WIN")
                win += 1
            else:
                print("LOOSE")
                loose += 1
print("WIN =",win,"\nLOOSE =",loose)
            

WIN
WIN
WIN
LOOSE
WIN
WIN = 4 
LOOSE = 1


In [46]:
## game 13:38

data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ 'Rhοbalas PXXL' +'?api_key='+api_key).json()
puuid_summoner = data_summoner.get("puuid")
id_summoner = data_summoner.get("id")

print("puuid =", puuid_summoner, "\nid =", id_summoner)

## récupère les matchs (les derniers) du summoner (ici 5 matchs)
## ressource utile les ID des queues : https://static.developer.riotgames.com/docs/lol/queues.json
## récupère la division, le tier et les LP du compte

matches_id = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid_summoner +'/ids?queue='+'420'+'&start=0&count=40&api_key='+api_key).json()
data_rank = requests.get('https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/'+ id_summoner +'?api_key='+api_key).json()
print(data_rank[0]['tier'],data_rank[0]['rank']," : ",data_rank[0]['leaguePoints']) 


puuid = 7BT9qPlBVWKxbxcDPlXgBTNCyQJgj4Tq4Y0ucZWZS2FBPhzH-ZkF9uUx_FfwxLUuD2Zey0dX9Jk4hw 
id = fixW7VXZS1rhLOaYStQDxBeo3KEfpUzuUi_cJ6Sd7PAx5CE
MASTER I  :  394


In [47]:
## récupère puis stocke le PUUID du summoner

data_summoner = requests.get('https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+ 'Rhοbalas PXXL' +'?api_key='+api_key).json()
puuid_summoner = data_summoner.get("puuid")
id_summoner = data_summoner.get("id")
#print("puuid =", puuid_summoner, "\nid =", id_summoner)

## récupère les matchs (les derniers) du summoner (ici 5 matchs)
## ressource utile les ID des queues : https://static.developer.riotgames.com/docs/lol/queues.json
## récupère la division, le tier et les LP du compte

matches_id = requests.get('https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+ puuid_summoner +'/ids?queue='+'420'+'&start=0&count=40&api_key='+api_key).json()
data_rank = requests.get('https://euw1.api.riotgames.com/lol/league/v4/entries/by-summoner/'+ id_summoner +'?api_key='+api_key).json()
print(data_rank[0]['tier'],data_rank[0]['rank']," : ",data_rank[0]['leaguePoints']) 


MASTER I  :  418
